In [7]:
# Conduct Voronoi and otsu segmentation

import sys
import cv2
from matplotlib import pyplot as plt
import pyclesperanto_prototype as cle
from skimage import io
import pandas as pd
import os

def main():
    
    outPath = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Measurements/Voronoi/"
    path = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/T30_Mock_0018/"
    
    data = []
    image = []

    # iterate through the images of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the image
        input_path = os.path.join(path, image_path)
        image_to_segment = io.imread(input_path)
        
        # step 1: heavy gaussian blurring to detect maxima (closer objects require decreased blurring)
        img_gaussian = cle.gaussian_blur(image_to_segment, sigma_x=3, sigma_y=3, sigma_z=3)
        img_maxima_locations = cle.detect_maxima_box(img_gaussian, radius_x=0, radius_y=0, radius_z=0)
        number_of_maxima_locations = cle.sum_of_all_pixels(img_maxima_locations)

        # step 2:threshold image after applying light gaussian blur
        img_gaussian2 = cle.gaussian_blur(image_to_segment, sigma_x=0.5, sigma_y=0.5, sigma_z=0.5)
        img_thresh = cle.threshold_otsu(img_gaussian2)
        
        # step 3: exclude maxima locations from the background
        img_relevant_maxima = cle.binary_and(img_thresh, img_maxima_locations)
        number_of_relevant_maxima_locations = cle.sum_of_all_pixels(img_relevant_maxima)

        # step 4: separate maximal locations into labels
        voronoi_separation = cle.masked_voronoi_labeling(img_relevant_maxima, img_thresh)
        
        # save count measurements
        image.append(image_path)
        data.append(number_of_relevant_maxima_locations)

    # export measurements
    df = pd.DataFrame(list(zip(image , data)), columns =['Image', 'Number of lamellipodia'])
    basename_without_ext = os.path.basename(os.path.dirname(path)) 
    df.to_excel(outPath + basename_without_ext + "_Voronoi.xlsx", index=False)
    
if __name__ == '__main__':
    main()

In [8]:
# calculate average values

import pandas as pd

def main():
    
    outPath = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Measurements/Voronoi/"
    path = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Measurements/Voronoi/"
    
    folder = []
    x = []
    s = []
    
    # iterate through excel files of the folder
    for file in os.listdir(path):

        # create the full input path and read the image
        input_path = os.path.join(path, file)
        df = pd.read_excel(input_path)
        
        # calculate mean
        mean = df["Number of lamellipodia"].mean()
        se = df["Number of lamellipodia"].sem()
        
        # save values
        folder.append(file)
        x.append(mean)
        s.append(se)
    
    # export values
    df = pd.DataFrame(list(zip(folder, x, s)), columns = ['File', 'Average number of lamellipodia', 'Standard error'])
    df.to_excel(outPath + "Voronoi_Compiled_Results.xlsx", index=False)
                    
if __name__ == '__main__':
    main()